In [1]:
import numpy as np
import os
import time

np.set_printoptions(threshold=np.inf)

In [2]:
def input(fname):
    fname = os.path.join(fname)

    data = []
    with open(fname) as f:
        for line in f:
            data.append(line.strip())
    return data  

In [ ]:
def hex2bin(data):
    h2b = lambda h: (bin(int(h, 16))[2:]).zfill(4)
    return ''.join([''.join(str(h2b(h))) for h in data])

# class Packet:
#     def __init__(self, data):
#         self.packet = data
#         self._extract()

#     def _extract(self):
#         self.data = self.packet
#         self._version()
#         self._type_ID()

#     def _version(self):
#         self.version = int(self.data[0:3],2)
#         self.data = self.data[3:]        

#     def _type_ID(self):
#         self.type_ID = int(self.data[0:3],2)
#         self.data = self.data[3:]        
    
# # class BITS:
# #     def __init__(self, packet):
# #         self.packet = packet
# #         self.decoded = []

#     # def read_literal(self, n=5):
#     #     lit = ''
#     #     for i in range (ceil(len(self.packet.data)/5)):
#     #         c = self.packet.data[n*i:n*i+5]
#     #         lit += c[1:n]
#     #         if c[0] == '0':
#     #             junk = self.packet.data[n*i+5:]
#     #             break
#     #     lit = int(lit,2)

#     #     self.decoded.append(lit)
        
#     #     return junk        

#     # def decode_packet(self, packet):
#     #     decoded = []
#     #     junk = None
#     #     # version, type_ID, data = extract_data(packet,is_hex)
#     #     if packet.type_ID == 4:
#     #         junk = read_literal(packet.data)
#     #     else:
#     #         length_type_ID = data[0]
#     #         data = data[1:]
#     #         if length_type_ID=='0':
#     #             sub_length = int(data[:15],2)
#     #             data = data[15:15+sub_length]
#     #             flag = True
#     #             while flag:
#     #                 _, _, lit, data = decode_packet(data,is_hex=False)
#     #                 decoded.extend(lit)
#     #                 if sum([int(i) for i in data])==0:
#     #                     flag = False
#     #                     junk = data
#     #         else:
#     #             num_subs = int(data[:11],2)
#     #             data = data[11:]
#     #             for i in range(num_subs):
#     #                 # print(len(data))
#     #                 # print(data)
#     #                 _, _, lit, data = decode_packet(data,is_hex=False)
#     #                 decoded.extend(lit)
                
#     #     return decoded, junk


# # packet = input('input.txt')
# packet = 'D2FE28'
# # packet = '38006F45291200'
# # packet = 'EE00D40C823060'

# # packet = '8A004A801A8002F478'             # YES, runs  - O(4) [ O(1) [O(5) L(6) ] ]
# # packet = '620080001611562C8802118E34'     # NO,  crash - O(3) [ O() [L(), L()], O() [L(), L()] ]
# # packet = 'C0015000016115A2E0802F182340'   # NO,  runs  - O()  [ O() [L(), L()], O() [L(), L()] ]
# # packet = 'A0016C880162017C3686B18A3D4780' # YES, crash - O()  [ O() [ O() [L(), L(), L(), L(), L(), ] ] ]
# class BITS:
#     def __init__(self, packet):
#         self.packet = Packet(packet)
#         self.decoded = []

# # data = Packet(hex2bin(packet))
# decoded, junk = BITS(packet)

# # print('V:', version)
# # print('T:', type_ID)
# # print('L:', decoded)
# # print('X:', junk)
# # print(sub_length)

# # 011 000 1 00000000010 | 00000000000000000101100001000101010110001011001000100000000010000100011000111000110100 
# #     000 000 0 000000000010110 0001000101010110001011001000100000000010000100011000111000110100
# #         000 100 0,1010 1,0110 0,01011
# #     001 000 1 00000000010 000100011000111000110100 
# #         000 100 0,1100 0,1110 00110100
# #             001 101 00

In [5]:
op_map = {
    0: 'sum',
    1: 'prod',
    2: 'min',
    3: 'max',
    5: 'gt',
    6: 'lt',
    7: 'eq',
}

def extract_data(packet, is_hex):
    if is_hex:
        p = ''
        for h in packet:
            p += str((bin(int(h, 16))[2:]).zfill(4))
    else:
        p = packet
    version = int(p[0:3],2)
    # print(version)
    p = p[3:]
    type_ID = int(p[0:3],2)
    p = p[3:]

    return version, type_ID, p

def read_literal(data, n=5):
    lit = ''
    reading = True
    while reading:
        c = data[:n]
        data = data[n:]
        lit += c[1:n]
        if c[0]=='0':
            reading = False
    return int(lit,2), data

def decode_packet(packet, is_hex=True):
    decoded = []
    rpn = []
    version = None
    type_ID = None

    version, type_ID, data = extract_data(packet,is_hex)
    # print('+',len(packet),sum([int(i) for i in data]),data)
    # print('+',len(packet),sum([int(i) for i in data]))
    vers.append(version)
    if type_ID != 4:
        type_ids.append(type_ID)
        rpn.append(op_map[type_ID])

    # if literal packet
    if type_ID == 4:
        lit, data = read_literal(data)
        # print('LITERAL:',lit,len(data))#,data)
        decoded.append(lit)
        rpn.append(lit)
    # if operator
    else:
        length_type_ID = data[0]
        data = data[1:]

        if (length_type_ID=='0'):
            # print('OP 0')
            length = int(data[:15],2)
            data = data[15:]
            # print('==',length)
            while (length > 0):
                l_before = len(data)
                lit, data, op = decode_packet(data,is_hex=False)
                length -= (l_before - len(data))
                # print('--',length)
                decoded.extend(lit)
                rpn.extend([op])
        else:
            # print('OP 1')
            num_subs = int(data[:11],2)
            data = data[11:]
            for _ in range(num_subs):
                # print(data)
                lit, data, op = decode_packet(data,is_hex=False)
                # print('...',len(data))
                decoded.extend(lit)
                rpn.extend([op])

    return decoded, data, rpn

packet = input('input.txt')[0]
test_input = [
    packet
    # 'D2FE28',
    # '38006F45291200',
    # 'EE00D40C823060',
    # '8A004A801A8002F478',
    # '620080001611562C8802118E34',
    # 'C0015000016115A2E0802F182340',
    # 'A0016C880162017C3686B18A3D4780',
    # 'C200B40A82',
    # '04005AC33890',
    # '880086C3E88112',
    # 'CE00C43D881120',
    # 'D8005AC2A8F0',
    # 'F600BC2D8F',
    # '9C005AC2F8F0',
    # '9C0141080250320F1802104A08'
    ]
for p in test_input:
    type_ids = []
    vers = []
    decoded, junk, rpn = decode_packet(p)
    print('---------------------')
    # print(decoded)
    # print(type_ids)
    # print(rpn)
    # print(vers)
    # print(sum(vers))
    res = calc_op(rpn)
    print(res)
    print('----')

---------------------
[1673210814091]
----


In [4]:
def calc_op(rpn):
    stack = []
    while len(rpn)>0:
        x = rpn.pop()
        # print(x,stack)
        if type(x) == list:
            c = calc_op(x)
            if type(c) == list:
                stack.extend(c)
            else:
                stack.append(c)
        if type(x) == int:
            stack.append(x)
        if type(x) == str:
            if x == 'sum':
                stack = [sum(stack)]
            elif x == 'prod':
                stack = [np.prod(stack)]
            elif x == 'min':
                stack = [min(stack)]
            elif x == 'max':
                stack = [max(stack)]
            elif x == 'gt':
                stack = [1] if stack[0]<stack[1] else 0
            elif x == 'lt':
                stack = [1] if stack[0]>stack[1] else 0
            elif x == 'eq':
                stack = [1] if stack[0]==stack[1] else 0
        # print(x, stack)
    return stack
        
    
